# with Groq Model

In [ ]:
#! touch "/content/drive/MyDrive/Colab Notebooks/OSINT_agentic/config/agents.yaml"

In [ ]:
!pip install -r "/content/drive/MyDrive/Colab Notebooks/OSINT_agentic/requirements.txt"

In [18]:
from google.colab import userdata
grq_api = userdata.get('grog_api')
serper_api = userdata.get('serper_api')
mistral_api = userdata.get('mistral_api')
anthropic_api = userdata.get('anthropic_api')

In [ ]:
from groq import Groq
client = Groq(api_key=grq_api)

try:
    # Send prompt to Groq API
    completion = client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[{"role": "user", "content": "are you free to use ? what is the usage limit for you?"}],
        temperature=0.7,
        max_completion_tokens=200,
        top_p=1,
        stream=True,
        stop=None,
    )

    # Stream output
    print("Response: ", end="")
    for chunk in completion:
        print(chunk.choices[0].delta.content or "", end="")

except Exception as e:
    print(f"\nError during API request: {e}")

Response: I'm a large language model, and my usage is governed by several factors. Here are some details:

1. **Fair use policy**: My usage is governed by a fair use policy, which means I can be used for personal, educational, or research purposes. However, I may not be used for commercial purposes without permission from my developers.
2. **Query limits**: I don't have a strict query limit, but I'm designed to handle a large volume of conversations. If I detect that you're making an excessive number of requests, I may slow down or limit my responses.
3. **Session limits**: I can maintain multiple conversations simultaneously, but each session has a limit on the number of turns (i.e., back-and-forth conversations). If we reach this limit, I may ask you to start a new conversation or take a break.
4. **Content limits**: I can process and respond to a wide range of content, including text, images, and other media. However

In [2]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

# # Load environment variables
# from helper import load_env
# load_env()

import os
import yaml
from crewai import Agent, Task, Crew

In [3]:
from pydantic import BaseModel, Field
from typing import List, Optional

class MalwareSample(BaseModel):
    hash: str = Field(..., description="The hash of the malware sample (e.g., SHA-256, MD5).")
    source: str = Field(..., description="The source from where the malware intelligence was gathered.")
    classification: Optional[str] = Field(None, description="The classification of the malware (packer, dropper, loader).")
    description: Optional[str] = Field(None, description="Brief description of the malware behavior.")
    threat_level: Optional[str] = Field(None, description="Severity of the malware (Low, Medium, High).")

class OSINTReport(BaseModel):
    collected_samples: List[MalwareSample] = Field(..., description="A list of collected malware samples with metadata.")
    intelligence_sources: List[str] = Field(..., description="Sources used for gathering information.")

class ThreatAnalysis(BaseModel):
    indicators_of_compromise: List[str] = Field(..., description="List of IOCs such as IPs, domains, file hashes, and URLs.")
    behavior_analysis: str = Field(..., description="Detailed behavior analysis of the malware.")
    attack_techniques: List[str] = Field(..., description="MITRE ATT&CK techniques associated with the malware.")

class ClassificationReport(BaseModel):
    categorized_samples: List[MalwareSample] = Field(..., description="List of malware samples with classification details.")
    classification_summary: str = Field(..., description="Summary of classification criteria used.")

class ResearchSummary(BaseModel):
    key_findings: str = Field(..., description="High-level summary of all gathered intelligence and analysis.")
    recommended_next_steps: List[str] = Field(..., description="Suggested next steps for research and further analysis.")
    safe_analysis_guidelines: str = Field(..., description="Best practices for safely handling and analyzing the malware.")

class MalwareAnalysisOutput(BaseModel):
    osint_report: OSINTReport = Field(..., description="OSINT intelligence gathering report.")
    threat_analysis: ThreatAnalysis = Field(..., description="Technical analysis of the malware.")
    classification_report: ClassificationReport = Field(..., description="Categorization of the malware.")
    research_summary: ResearchSummary = Field(..., description="Final research findings and recommendations.")


In [ ]:
import yaml
from crewai import Agent, Task, Crew
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, WebsiteSearchTool
os.environ['GROQ_API_KEY'] = grq_api
os.environ['SERPER_API_KEY'] = serper_api

# Define Groq LLM model
groq_llm = "groq/llama-3.1-8b-instant"

# Define file paths for YAML configurations
files = {
    'agents': '/content/drive/MyDrive/Colab Notebooks/OSINT_agentic/config/agents.yaml',
    'tasks': '/content/drive/MyDrive/Colab Notebooks/OSINT_agentic/config/tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs['agents']
tasks_config = configs['tasks']

# Creating Agents
osint_collector_agent = Agent(
    config=agents_config['osint_collector_agent'],
    tools=[SerperDevTool()],
    llm=groq_llm,
    max_iter=3,
    max_rpm=3,
    max_execution_time=30,
    verbose=True,
    max_retry_limit=1,
    respect_context_window=True,
    allow_delegation=False
)

threat_analyst_agent = Agent(
    config=agents_config['threat_analyst_agent'],
    tools=[SerperDevTool()],
    llm=groq_llm,
    max_iter=3,
    max_rpm=3,
    max_execution_time=30,
    verbose=True,
    max_retry_limit=1,
    respect_context_window=True,
    allow_delegation=False
)

malware_categorizer_agent = Agent(
    config=agents_config['malware_categorizer_agent'],
    tools=[ScrapeWebsiteTool()],
    llm=groq_llm,
    max_iter=3,
    max_rpm=3,
    max_execution_time=30,
    verbose=True,
    max_retry_limit=1,
    respect_context_window=True,
    allow_delegation=False
)

research_advisor_agent = Agent(
    config=agents_config['research_advisor_agent'],
    tools=[],
    llm=groq_llm,
    max_iter=3,
    max_rpm=3,
    max_execution_time=30,
    verbose=True,
    max_retry_limit=1,
    respect_context_window=True,
    allow_delegation=False
)

# Creating Tasks
search_malware_intelligence_task = Task(
    config=tasks_config['search_malware_intelligence'],
    agent=osint_collector_agent
)

analyze_malware_data_task = Task(
    config=tasks_config['analyze_malware_data'],
    agent=threat_analyst_agent
)

classify_malware_type_task = Task(
    config=tasks_config['classify_malware_type'],
    agent=malware_categorizer_agent,
    context=[search_malware_intelligence_task, analyze_malware_data_task]
)

generate_research_summary_task = Task(
    config=tasks_config['generate_research_summary'],
    agent=research_advisor_agent,
    context=[search_malware_intelligence_task, analyze_malware_data_task, classify_malware_type_task],
    output_pydantic=MalwareAnalysisOutput
)

# Creating Crew
malware_analysis_crew = Crew(
    agents=[
        osint_collector_agent,
        threat_analyst_agent,
        malware_categorizer_agent,
        research_advisor_agent
    ],
    tasks=[
        search_malware_intelligence_task,
        analyze_malware_data_task,
        classify_malware_type_task,
        generate_research_summary_task
    ],
    verbose=True
)

# Run the crew
if __name__ == "__main__":
    results = malware_analysis_crew.kickoff(inputs={'topic': 'find malwares like packers,droppers and loaders from sources like VirusTotal, Hybrid Analysis, MITRE ATT&CK, and underground forums'})




# Agent: OSINT Researcher
## Task: Conduct an extensive OSINT search for information on malware families,  focusing on packers, droppers, and loaders. Gather data from sources like  VirusTotal, Hybrid Analysis, MITRE ATT&CK, and cybersecurity research forums.

 

I encountered an error while trying to use the tool. This was the error: Arguments validation failed: 1 validation error for SerperDevToolSchema
search_query
  Input should be a valid string [type=string_type, input_value={'description': 'packers ...ers loaders VirusTotal'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/string_type.
 Tool Search the internet with Serper accepts these inputs: Tool Name: Search the internet with Serper
Tool Arguments: {'search_query': {'description': 'Mandatory search query you want to use to search the internet', 'type': 'str'}}
Tool Description: A tool that can be used to search the internet with a search_query. Supports different search types: 'search' 

ERROR:root:LiteLLM call failed: litellm.RateLimitError: RateLimitError: GroqException - {"error":{"message":"Rate limit reached for model `llama-3.1-8b-instant` in organization `org_01jnmhpvvbenwrkhevabm1d2g0` service tier `on_demand` on tokens per minute (TPM): Limit 6000, Used 4922, Requested 2967. Please try again in 18.881s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing","type":"tokens","code":"rate_limit_exceeded"}}





LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.


Provider List: https://docs.litellm.ai/docs/providers

 Error during LLM call: litellm.RateLimitError: RateLimitError: GroqException - {"error":{"message":"Rate limit reached for model `llama-3.1-8b-instant` in organization `org_01jnmhpvvbenwrkhevabm1d2g0` service tier `on_demand` on tokens per minute (TPM): Limit 6000, Used 4922, Requested 2967. Please try again in 18.881s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing","type":"tokens","code":"rate_limit_exceeded"}}



RateLimitError: litellm.RateLimitError: RateLimitError: GroqException - {"error":{"message":"Rate limit reached for model `llama-3.1-8b-instant` in organization `org_01jnmhpvvbenwrkhevabm1d2g0` service tier `on_demand` on tokens per minute (TPM): Limit 6000, Used 4922, Requested 2967. Please try again in 18.881s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing","type":"tokens","code":"rate_limit_exceeded"}}


In [ ]:
import yaml
from crewai import Agent, Task, Crew
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, WebsiteSearchTool
os.environ['GROQ_API_KEY'] = grq_api
os.environ['SERPER_API_KEY'] = serper_api

# Define Groq LLM model
groq_llm = "groq/llama-3.1-8b-instant"

# Define file paths for YAML configurations
files = {
    'agents': '/content/drive/MyDrive/Colab Notebooks/OSINT_agentic/config/agents.yaml',
    'tasks': '/content/drive/MyDrive/Colab Notebooks/OSINT_agentic/config/tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs['agents']
tasks_config = configs['tasks']

# Creating Agents
osint_collector_agent = Agent(
    config=agents_config['osint_collector_agent'],
    tools=[SerperDevTool()],
    llm=groq_llm,
    max_iter=3,
    max_rpm=3,
    max_execution_time=30,
    verbose=True,
    max_retry_limit=1,
    respect_context_window=True,
    allow_delegation=False
)

threat_analyst_agent = Agent(
    config=agents_config['threat_analyst_agent'],
    tools=[SerperDevTool()],
    llm=groq_llm,
    max_iter=3,
    max_rpm=3,
    max_execution_time=30,
    verbose=True,
    max_retry_limit=1,
    respect_context_window=True,
    allow_delegation=False
)

malware_categorizer_agent = Agent(
    config=agents_config['malware_categorizer_agent'],
    tools=[ScrapeWebsiteTool()],
    llm=groq_llm,
    max_iter=3,
    max_rpm=3,
    max_execution_time=30,
    verbose=True,
    max_retry_limit=1,
    respect_context_window=True,
    allow_delegation=False
)

research_advisor_agent = Agent(
    config=agents_config['research_advisor_agent'],
    tools=[],
    llm=groq_llm,
    max_iter=3,
    max_rpm=3,
    max_execution_time=30,
    verbose=True,
    max_retry_limit=1,
    respect_context_window=True,
    allow_delegation=False
)

# Creating Tasks
search_malware_intelligence_task = Task(
    config=tasks_config['search_malware_intelligence'],
    agent=osint_collector_agent
)

analyze_malware_data_task = Task(
    config=tasks_config['analyze_malware_data'],
    agent=threat_analyst_agent
)

classify_malware_type_task = Task(
    config=tasks_config['classify_malware_type'],
    agent=malware_categorizer_agent,
    context=[search_malware_intelligence_task, analyze_malware_data_task]
)

generate_research_summary_task = Task(
    config=tasks_config['generate_research_summary'],
    agent=research_advisor_agent,
    context=[search_malware_intelligence_task, analyze_malware_data_task, classify_malware_type_task],
    output_pydantic=MalwareAnalysisOutput
)

# Creating Crew
malware_analysis_crew = Crew(
    agents=[
        osint_collector_agent,
        threat_analyst_agent,
        malware_categorizer_agent,
        research_advisor_agent
    ],
    tasks=[
        search_malware_intelligence_task,
        analyze_malware_data_task,
        classify_malware_type_task,
        generate_research_summary_task
    ],
    verbose=True
)

# Run the crew
if __name__ == "__main__":
    results = malware_analysis_crew.kickoff(inputs={'topic': 'find malwares like packers,droppers and loaders from sources like VirusTotal, Hybrid Analysis, MITRE ATT&CK, and underground forums'})




/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_config.py:295: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
/usr/local/lib/python3.11/dist-packages/crewai_tools/tools/scrapegraph_scrape_tool/scrapegraph_scrape_tool.py:34: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  @validator("website_url")
/usr/local/lib/python3.11/dist-packages/crewai_tools/tools/selenium_scraping_tool/selenium_scraping_tool.py:26: PydanticDeprecatedSince20: Pydantic V1 style `@validator`

# Agent: find malwares like packers,droppers and loaders from sources like VirusTotal, Hybrid Analysis, MITRE ATT&CK, and underground forums OSINT Researcher
## Task: Conduct an extensive OSINT search for information on malware families,  focusing on packers, droppers, and loaders. Gather data from sources like  VirusTotal, Hybrid Analysis, MITRE ATT&CK, and cybersecurity research forums.



# Agent: find malwares like packers,droppers and loaders from sources like VirusTotal, Hybrid Analysis, MITRE ATT&CK, and underground forums OSINT Researcher
## Thought: Thought: to gather the necessary information on packers, droppers, and loaders, I should start by searching for known malware families and samples using trusted sources like VirusTotal and Hybrid Analysis.
## Using tool: Search the internet with Serper
## Tool Input: 
"{\"search_query\": \"{\\\"keyword\\\": \\\"packers malware family\\\"}\"}"
## Tool Output: 
{'searchParameters': {'q': '{"keyword": "packers malware family"}', 'type

ERROR:root:LiteLLM call failed: litellm.RateLimitError: RateLimitError: GroqException - {"error":{"message":"Rate limit reached for model `llama-3.1-8b-instant` in organization `org_01jnmhpvvbenwrkhevabm1d2g0` service tier `on_demand` on tokens per minute (TPM): Limit 6000, Used 5472, Requested 818. Please try again in 2.893999999s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing","type":"tokens","code":"rate_limit_exceeded"}}





LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.


Provider List: https://docs.litellm.ai/docs/providers

 Error during LLM call: litellm.RateLimitError: RateLimitError: GroqException - {"error":{"message":"Rate limit reached for model `llama-3.1-8b-instant` in organization `org_01jnmhpvvbenwrkhevabm1d2g0` service tier `on_demand` on tokens per minute (TPM): Limit 6000, Used 5472, Requested 818. Please try again in 2.893999999s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing","type":"tokens","code":"rate_limit_exceeded"}}



RateLimitError: litellm.RateLimitError: RateLimitError: GroqException - {"error":{"message":"Rate limit reached for model `llama-3.1-8b-instant` in organization `org_01jnmhpvvbenwrkhevabm1d2g0` service tier `on_demand` on tokens per minute (TPM): Limit 6000, Used 5472, Requested 818. Please try again in 2.893999999s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing","type":"tokens","code":"rate_limit_exceeded"}}


In [ ]:
import time
from functools import wraps
import litellm  # Ensure you have this installed: pip install litellm
from crewai import LLM

def retry_with_exponential_backoff(func, max_retries=7, initial_delay=3):
    """Retries LLM requests with an increasing backoff delay."""
    @wraps(func)
    def wrapper(*args, **kwargs):
        delay = initial_delay
        for attempt in range(max_retries):
            try:
                return func(*args, **kwargs)
            except litellm.RateLimitError as e:
                print(f"Rate limit exceeded. Retrying in {delay} seconds... (Attempt {attempt+1}/{max_retries})")
                if attempt == max_retries - 1:
                    raise  # Give up after max retries
                time.sleep(delay)
                delay *= 2  # Exponential backoff
    return wrapper


In [ ]:
class RetriableLLM(LLM):
    """LLM wrapper to include retry logic on completions."""

    @retry_with_exponential_backoff
    def complete(self, prompt: str):
        return super().complete(prompt)


In [ ]:


# Instantiate the LLM with retry logic
retriable_llm = RetriableLLM(model="groq/llama-3.1-8b-instant")



In [ ]:
import yaml
from crewai import Agent, Task, Crew
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, WebsiteSearchTool
os.environ['GROQ_API_KEY'] = grq_api
os.environ['SERPER_API_KEY'] = serper_api

# Define file paths for YAML configurations
files = {
    'agents': '/content/drive/MyDrive/Colab Notebooks/OSINT_agentic/config/agents.yaml',
    'tasks': '/content/drive/MyDrive/Colab Notebooks/OSINT_agentic/config/tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs['agents']
tasks_config = configs['tasks']

# Creating Agents
osint_collector_agent = Agent(
    config=agents_config['osint_collector_agent'],
    tools=[SerperDevTool()],
    llm=retriable_llm
)

threat_analyst_agent = Agent(
    config=agents_config['threat_analyst_agent'],
    tools=[SerperDevTool()],
    llm=retriable_llm
)

malware_categorizer_agent = Agent(
    config=agents_config['malware_categorizer_agent'],
    tools=[ScrapeWebsiteTool()],
    llm=retriable_llm
)

research_advisor_agent = Agent(
    config=agents_config['research_advisor_agent'],
    tools=[],
    llm=retriable_llm
)

# Creating Tasks
search_malware_intelligence_task = Task(
    config=tasks_config['search_malware_intelligence'],
    agent=osint_collector_agent
)

analyze_malware_data_task = Task(
    config=tasks_config['analyze_malware_data'],
    agent=threat_analyst_agent
)

classify_malware_type_task = Task(
    config=tasks_config['classify_malware_type'],
    agent=malware_categorizer_agent,
    context=[search_malware_intelligence_task, analyze_malware_data_task]
)

generate_research_summary_task = Task(
    config=tasks_config['generate_research_summary'],
    agent=research_advisor_agent,
    context=[search_malware_intelligence_task, analyze_malware_data_task, classify_malware_type_task],
    output_pydantic=MalwareAnalysisOutput
)

# Creating Crew
malware_analysis_crew = Crew(
    agents=[
        osint_collector_agent,
        threat_analyst_agent,
        malware_categorizer_agent,
        research_advisor_agent
    ],
    tasks=[
        search_malware_intelligence_task,
        analyze_malware_data_task,
        classify_malware_type_task,
        generate_research_summary_task
    ],
    verbose=True
)

# Run the crew
if __name__ == "__main__":
    results = malware_analysis_crew.kickoff(inputs={'topic': 'find malwares like packers,droppers and loaders from sources like VirusTotal, Hybrid Analysis, MITRE ATT&CK, and underground forums'})




# Agent: find malwares like packers,droppers and loaders from sources like VirusTotal, Hybrid Analysis, MITRE ATT&CK, and underground forums OSINT Researcher
## Task: Conduct an extensive OSINT search for information on malware families,  focusing on packers, droppers, and loaders. Gather data from sources like  VirusTotal, Hybrid Analysis, MITRE ATT&CK, and cybersecurity research forums.



# Agent: find malwares like packers,droppers and loaders from sources like VirusTotal, Hybrid Analysis, MITRE ATT&CK, and underground forums OSINT Researcher
## Using tool: Search the internet with Serper
## Tool Input: 
"{\"search_query\": \"packers droppers and loaders malware intelligence report\", \"search_type\": \"search\"}"
## Tool Output: 
{'searchParameters': {'q': 'packers droppers and loaders malware intelligence report', 'type': 'search', 'num': 10, 'engine': 'google'}, 'organic': [{'title': 'Explained: Packer, Crypter, and Protector - ThreatDown', 'link': 'https://www.threatdown.com/bl

ERROR:root:LiteLLM call failed: litellm.RateLimitError: RateLimitError: GroqException - {"error":{"message":"Rate limit reached for model `llama-3.1-8b-instant` in organization `org_01jnmhpvvbenwrkhevabm1d2g0` service tier `on_demand` on tokens per minute (TPM): Limit 6000, Used 4493, Requested 2840. Please try again in 13.328s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing","type":"tokens","code":"rate_limit_exceeded"}}





LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.


Provider List: https://docs.litellm.ai/docs/providers

 Error during LLM call: litellm.RateLimitError: RateLimitError: GroqException - {"error":{"message":"Rate limit reached for model `llama-3.1-8b-instant` in organization `org_01jnmhpvvbenwrkhevabm1d2g0` service tier `on_demand` on tokens per minute (TPM): Limit 6000, Used 4493, Requested 2840. Please try again in 13.328s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing","type":"tokens","code":"rate_limit_exceeded"}}



RateLimitError: litellm.RateLimitError: RateLimitError: GroqException - {"error":{"message":"Rate limit reached for model `llama-3.1-8b-instant` in organization `org_01jnmhpvvbenwrkhevabm1d2g0` service tier `on_demand` on tokens per minute (TPM): Limit 6000, Used 4493, Requested 2840. Please try again in 13.328s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing","type":"tokens","code":"rate_limit_exceeded"}}


#with mistral model

In [17]:
import yaml
from crewai import Agent, Task, Crew, LLM
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, WebsiteSearchTool
os.environ['MISTRAL_API_KEY'] = mistral_api
os.environ['SERPER_API_KEY'] = serper_api


# Define file paths for YAML configurations
files = {
    'agents': '/content/drive/MyDrive/Colab Notebooks/OSINT_agentic/config/agents.yaml',
    'tasks': '/content/drive/MyDrive/Colab Notebooks/OSINT_agentic/config/tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs['agents']
tasks_config = configs['tasks']
# retriable_llm = LLM(provider="mistral", model="mistral-large-latest", api_key=os.getenv("MISTRAL_API_KEY"))
retriable_llm = LLM(
    model="mistral/mistral-medium",
    temperature=0.7
)
#retriable_llm = "mistral/mistral-large-latest"

# Creating Agents
osint_collector_agent = Agent(
    config=agents_config['osint_collector_agent'],
    tools=[SerperDevTool()],
    llm=retriable_llm
)

threat_analyst_agent = Agent(
    config=agents_config['threat_analyst_agent'],
    tools=[SerperDevTool()],
    llm=retriable_llm
)

malware_categorizer_agent = Agent(
    config=agents_config['malware_categorizer_agent'],
    tools=[ScrapeWebsiteTool()],
    llm=retriable_llm
)

research_advisor_agent = Agent(
    config=agents_config['research_advisor_agent'],
    tools=[],
    llm=retriable_llm
)

# Creating Tasks
search_malware_intelligence_task = Task(
    config=tasks_config['search_malware_intelligence'],
    agent=osint_collector_agent
)

analyze_malware_data_task = Task(
    config=tasks_config['analyze_malware_data'],
    agent=threat_analyst_agent
)

classify_malware_type_task = Task(
    config=tasks_config['classify_malware_type'],
    agent=malware_categorizer_agent,
    context=[search_malware_intelligence_task, analyze_malware_data_task]
)

generate_research_summary_task = Task(
    config=tasks_config['generate_research_summary'],
    agent=research_advisor_agent,
    context=[search_malware_intelligence_task, analyze_malware_data_task, classify_malware_type_task],
    output_pydantic=MalwareAnalysisOutput
)

# Creating Crew
malware_analysis_crew = Crew(
    agents=[
        osint_collector_agent,
        threat_analyst_agent,
        malware_categorizer_agent,
        research_advisor_agent
    ],
    tasks=[
        search_malware_intelligence_task,
        analyze_malware_data_task,
        classify_malware_type_task,
        generate_research_summary_task
    ],
    verbose=True
)

# Run the crew
if __name__ == "__main__":
    results = malware_analysis_crew.kickoff(inputs={'topic': 'find malwares like packers,droppers and loaders from sources like VirusTotal, Hybrid Analysis, MITRE ATT&CK, and underground forums'})




ERROR:opentelemetry.sdk.trace.export:Exception while exporting Span batch.
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py", line 464, in _make_request
    self._validate_conn(conn)
  File "/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py", line 1093, in _validate_conn
    conn.connect()
  File "/usr/local/lib/python3.11/dist-packages/urllib3/connection.py", line 741, in connect
    sock_and_verified = _ssl_wrap_socket_and_match_hostname(
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/urllib3/connection.py", line 920, in _ssl_wrap_socket_and_match_hostname
    ssl_sock = ssl_wrap_socket(
               ^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/urllib3/util/ssl_.py", line 460, in ssl_wrap_socket
    ssl_sock = _ssl_wrap_socket_impl(sock, context, tls_in_tls, server_hostname)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

# Agent: find malwares like packers,droppers and loaders from sources like VirusTotal, Hybrid Analysis, MITRE ATT&CK, and underground forums OSINT Researcher
## Task: Conduct an extensive OSINT search for information on malware families,  focusing on packers, droppers, and loaders. Gather data from sources like  VirusTotal, Hybrid Analysis, MITRE ATT&CK, and cybersecurity research forums.

 

I encountered an error while trying to use the tool. This was the error: Arguments validation failed: 1 validation error for SerperDevToolSchema
search_query
  Field required [type=missing, input_value={'search\\_query': 'packe...te:hybrid-analysis.com'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing.
 Tool Search the internet with Serper accepts these inputs: Tool Name: Search the internet with Serper
Tool Arguments: {'search_query': {'description': 'Mandatory search query you want to use to search the internet', 'type': 'str'}}
Tool Description: A 

ERROR:root:LiteLLM call failed: litellm.BadRequestError: MistralException - Error code: 400 - {'object': 'error', 'message': 'Expected last role User or Tool (or Assistant with prefix True) for serving but got assistant', 'type': 'invalid_request_error', 'param': None, 'code': None}




LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.


Provider List: https://docs.litellm.ai/docs/providers

 Error during LLM call: litellm.BadRequestError: MistralException - Error code: 400 - {'object': 'error', 'message': 'Expected last role User or Tool (or Assistant with prefix True) for serving but got assistant', 'type': 'invalid_request_error', 'param': None, 'code': None}


BadRequestError: litellm.BadRequestError: MistralException - Error code: 400 - {'object': 'error', 'message': 'Expected last role User or Tool (or Assistant with prefix True) for serving but got assistant', 'type': 'invalid_request_error', 'param': None, 'code': None}

#Anthropic model (Claude)

In [19]:
import yaml
from crewai import Agent, Task, Crew, LLM
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, WebsiteSearchTool
os.environ['ANTHROPIC_API_KEY'] = anthropic_api
os.environ['SERPER_API_KEY'] = serper_api


# Define file paths for YAML configurations
files = {
    'agents': '/content/drive/MyDrive/Colab Notebooks/OSINT_agentic/config/agents.yaml',
    'tasks': '/content/drive/MyDrive/Colab Notebooks/OSINT_agentic/config/tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs['agents']
tasks_config = configs['tasks']

retriable_llm = LLM(
    model="anthropic/claude-3-sonnet-20240229-v1:0",
    temperature=0.7
)


# Creating Agents
osint_collector_agent = Agent(
    config=agents_config['osint_collector_agent'],
    tools=[SerperDevTool()],
    llm=retriable_llm
)

threat_analyst_agent = Agent(
    config=agents_config['threat_analyst_agent'],
    tools=[SerperDevTool()],
    llm=retriable_llm
)

malware_categorizer_agent = Agent(
    config=agents_config['malware_categorizer_agent'],
    tools=[ScrapeWebsiteTool()],
    llm=retriable_llm
)

research_advisor_agent = Agent(
    config=agents_config['research_advisor_agent'],
    tools=[],
    llm=retriable_llm
)

# Creating Tasks
search_malware_intelligence_task = Task(
    config=tasks_config['search_malware_intelligence'],
    agent=osint_collector_agent
)

analyze_malware_data_task = Task(
    config=tasks_config['analyze_malware_data'],
    agent=threat_analyst_agent
)

classify_malware_type_task = Task(
    config=tasks_config['classify_malware_type'],
    agent=malware_categorizer_agent,
    context=[search_malware_intelligence_task, analyze_malware_data_task]
)

generate_research_summary_task = Task(
    config=tasks_config['generate_research_summary'],
    agent=research_advisor_agent,
    context=[search_malware_intelligence_task, analyze_malware_data_task, classify_malware_type_task],
    output_pydantic=MalwareAnalysisOutput
)

# Creating Crew
malware_analysis_crew = Crew(
    agents=[
        osint_collector_agent,
        threat_analyst_agent,
        malware_categorizer_agent,
        research_advisor_agent
    ],
    tasks=[
        search_malware_intelligence_task,
        analyze_malware_data_task,
        classify_malware_type_task,
        generate_research_summary_task
    ],
    verbose=True
)

# Run the crew
if __name__ == "__main__":
    results = malware_analysis_crew.kickoff(inputs={'topic': 'find malwares like packers,droppers and loaders from sources like VirusTotal, Hybrid Analysis, MITRE ATT&CK, and underground forums'})




ERROR:root:LiteLLM call failed: litellm.BadRequestError: AnthropicException - {"type":"error","error":{"type":"invalid_request_error","message":"Your credit balance is too low to access the Anthropic API. Please go to Plans & Billing to upgrade or purchase credits."}}


# Agent: find malwares like packers,droppers and loaders from sources like VirusTotal, Hybrid Analysis, MITRE ATT&CK, and underground forums OSINT Researcher
## Task: Conduct an extensive OSINT search for information on malware families,  focusing on packers, droppers, and loaders. Gather data from sources like  VirusTotal, Hybrid Analysis, MITRE ATT&CK, and cybersecurity research forums.



LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.


Provider List: https://docs.litellm.ai/docs/providers

 Error during LLM call: litellm.BadRequestError: AnthropicException - {"type":"error","error":{"type":"invalid_request_error","message":"Your credit balance is too low to access the Anthropic API. Please go to Plans & Billing to upgrade or purchase credits."}}


BadRequestError: litellm.BadRequestError: AnthropicException - {"type":"error","error":{"type":"invalid_request_error","message":"Your credit balance is too low to access the Anthropic API. Please go to Plans & Billing to upgrade or purchase credits."}}